In [8]:
def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array 
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    s = sudoku.copy()
    if not valid_sudoku_check(s):
        return np.array([np.array([-1 for x in range(0, 9)]) for y in range(0, 9)])
    grid = create_value_grid(s)
    naked_pairs(s, grid)

    if not solve(s, grid):
        return np.array([np.array([-1 for x in range(0, 9)]) for y in range(0, 9)])

    return s


def create_value_grid(sudoku):
    grid = []
    for r in range(0,9):
        row = []
        for c in range(0,9):
            if sudoku[r][c] != 0:
                values = {sudoku[r][c]}

            else:
                values = set()
                for i in range(1,10):
                    if is_valid_number(i, r, c, sudoku):
                        values.add(i)
            row.append(values)
        grid.append(row)
    return grid

def is_valid_number(n, r, c, sudoku):
    valid = False
    row_contents = sudoku[r]
    column_contents = [row[c] for row in sudoku]
    r_bound = (r // 3) * 3
    c_bound = (c // 3) * 3
    quadrant = []
    for x in range(r_bound, r_bound+3):
        for y in range(c_bound, c_bound+3):
            quadrant.append(sudoku[x][y])
    if n in quadrant or n in row_contents or n in column_contents:
        return False
    return True

def valid_sudoku_check(sudoku):  # check for duplicate values in columns, rows and *add quadrants*
    for row in sudoku:
        nums = set()
        for num in row:
            if num in nums and num != 0:
                return False
            else:
                nums.add(num)
                
    for c in range(0,9):
        nums = set()
        col = [row[c] for row in sudoku]
        for num in col:
            if num in nums and num != 0:
                return False
            else:
                nums.add(num)
    
    for i in range(0,9,3):
        for j in range(0,9,3):
            quad = [sudoku[x+i][y+j] for y in range(0,3) for x in range(0,3)]
            nums = set()
            for num in quad:
                if num in nums and num !=0:
                    return False
                else:
                    nums.add(num)        
    return True


# Find the cell with the least values
def decide_next_cell(g, s):
    smallest_length = 10
    for r in range(0,9):
        for c in range(0,9):
            if s[r][c] == 0:
                current_length = len(g[r][c])
                if current_length < smallest_length:
                    smallest_length = current_length
                    coords = (r, c)
    # If there are no zeros
    if smallest_length == 10:
        return None
    return coords


def solve(s, g):
    cell = decide_next_cell(g, s)
    if cell == None:
        return True
    r, c = cell[0], cell[1]
    values = lcv(s, g, r, c)
    r_bound = (r // 3) * 3
    c_bound = (c // 3) * 3
    for val in values:
        removed = []
        if is_valid_number(val, r, c, s):
            s[r][c] = val
            for i in range(0,9):
                if (r, i) == (r, c) or (i, c) == (r, c):
                    pass
                else:
                    if val in g[r][i]:
                        if len(g[r][i]) == 1:
                            s[r][c] = 0
                            for coord in removed:
                                g[coord[0]][coord[1]].add(val)
                            g[r][c] = set(values)
                            return False
                        g[r][i].remove(val)
                        removed.append((r,i))
                    if val in g[i][c]:
                        if len(g[i][c]) == 1:
                            s[r][c] = 0
                            for coord in removed:
                                g[coord[0]][coord[1]].add(val)
                            g[r][c] = set(values)
                            return False
                        g[i][c].remove(val)
                        removed.append((i,c))
                        
            for j in range(r_bound,r_bound+3, 2):
                for k in range(c_bound,c_bound+3, 2):
                    if (j, k) == (r, c):
                        pass
                    else:
                        if val in g[j][k]:
                            if len(g[j][k]) == 1:
                                s[r][c] = 0
                                for coord in removed:
                                    g[coord[0]][coord[1]].add(val)
                                g[r][c] = set(values)
                                return False
                            g[j][k].remove(val)
                            removed.append((j,k))
            if solve(s, g):
                return True
        s[r][c] = 0
        for coord in removed:
            g[coord[0]][coord[1]].add(val)
        g[r][c] = set(values)
    return False


def lcv(s, g, r, c):
    r_bound = (r // 3) * 3
    c_bound = (c // 3) * 3
    cells = [g[r][i] for i in range(0,9) if i != c] + [g[i][c] for i in range(9) if i != r] +[g[x][y] for x in range(r_bound, r_bound + 3, 2) for y in range(c_bound, c_bound + 3, 2) if (x, y) != (r, c)]
    counts = []
    for val in g[r][c]:
        count = 0
        for cell in cells:
            if val in cell:
                count+=1
        counts.append((val, count))
    counts.sort(key=lambda x: x[1])
    return [val[0] for val in counts]


def naked_pairs(sudoku, grid):
    flag_quad = False
    flag_line = False
    for x in range(0,8,3):
        for y in range(0,8,3):
            flag_quad = naked_pairs_quad(sudoku, grid, x, y)
    for i in range(0,8):
        flag_row = naked_pairs_line(sudoku, grid, x, y)
    

# change this to use dictionaries  
def naked_pairs_line(sudoku, grid, r, c):
    duos_row = []
    duos_col = []
    for y in range(8):
        if len(grid[r][y]) == 2:
            duos_row.append(((r,y), grid[r][y]))
            
    for x in range(8):
        if len(grid[x][c]) == 2:
            duos_col.append(((x,c), grid[x][c]))

    pairs_row = []
    for i in range(len(duos_row)):
        for j in range(i+1, len(duos_row)):
            if duos_row[i][1] == duos_row[j][1] and duos_row[i][0] != duos_row[j][0]:
                pairs_row.append(([duos_row[i][0], duos_row[j][0]], duos_row[i][1]))  # append tuple of coords and set

    pairs_col = []
    for i in range(len(duos_col)):
        for j in range(i+1, len(duos_col)):
            if duos_col[i][1] == duos_col[j][1] and duos_col[i][0] != duos_col[j][0]:
                pairs_col.append(([duos_col[i][0], duos_col[j][0]], duos_col[i][1]))  # append tuple of coords and set

    if pairs_row == [] and pairs_col == []:
        return False  
    

    removed_row = []
    for y in range(8):
        for pair in pairs_row:
            for val in pair[1]:
                if val in grid[r][y] and (r, y) not in pair[0]:
                    grid[r][y].remove(val)
                    removed_row.append(((r,y), val))
                if len(grid[r][y]) == 0 :
                    for rem in removed_row:
                        if rem[1] == val:
                            grid[rem[0][0]][rem[0][1]].add(rem[1])
                            removed_row.remove(rem)
    removed_col = []
    for x in range(8):
        for pair in pairs_col:
            for val in pair[1]:
                if val in grid[x][c] and (x, c) not in pair[0]:
                    grid[x][c].remove(val)
                    removed_col.append(((x,c), val))
                if len(grid[x][c]) == 0 :
                    for rem in removed_col:
                        if rem[1] == val:
                            grid[rem[0][0]][rem[0][1]].add(rem[1])
                            removed_col.remove(rem)
    if removed_row != [] or removed_col != []:
        return True
    else:
        return False
    
def naked_pairs_quad(sudoku, grid, r, c):
    duos_quad = []
    r_upper = (r // 3) * 3 + 3
    c_upper = (c // 3) * 3 +3
    for x in range(r_upper-3, r_upper):
        for y in range(c_upper-3, c_upper):
            if len(grid[x][y]) == 2:
                duos_quad.append(((x,y), grid[x][y]))  # append tuple of coord and set
    
    pairs_quad = []
    for i in range(len(duos_quad)):
        for j in range(i+1, len(duos_quad)):
            if duos_quad[i][1] == duos_quad[j][1] and duos_quad[i][0] != duos_quad[j][0]:
                pairs_quad.append(([duos_quad[i][0], duos_quad[j][0]], duos_quad[i][1]))  # append tuple of coords and set
    
    
    if pairs_quad == []:
        return False  
                
    removed_quad = []
    for x in range(r_upper-3, r_upper):
        for y in range(c_upper-3, c_upper):
            for pair in pairs_quad:
                for val in pair[1]:
                    if val in grid[x][y] and (x, y) not in pair[0]:
                        grid[x][y].remove(val)
                        removed_quad.append(((x,y), val))
                    if len(grid[x][y]) == 0 :
                        for rem in removed_quad:
                            if rem[1] == val:
                                grid[rem[0][0]][rem[0][1]].add(rem[1])
                                removed_quad.remove(rem)
    if removed_quad != []:
        return True
    else:
        return False

In [9]:
import time
import numpy as np
times = []

for i in range(0,100):
    start_time = time.process_time()
    s = np.load(f"hard_sudokus.npy")[i]
    j = sudoku_solver(s)
    end_time = time.process_time()
    if not valid_sudoku_check(j):
        raise ValueError(f'Failed {i}', j)
    print(f"{i} complete. time: {end_time-start_time}")
    for row in j:
        for col in row:
            if col == 0:
                raise ValueError('Oh no')
    times.append(end_time-start_time)

print('Average time to solve hard sudoku: ', sum(times)/len(times))

0 complete. time: 0.09453999999999851
1 complete. time: 0.005528000000001754
2 complete. time: 0.0036830000000023233
3 complete. time: 0.05149600000000021
4 complete. time: 0.003914999999999225
5 complete. time: 0.009281999999998902
6 complete. time: 7.1928090000000005
7 complete. time: 4.0255420000000015
8 complete. time: 0.00549999999999784
9 complete. time: 0.0037769999999994752
10 complete. time: 1.310144000000001
11 complete. time: 0.0038740000000032637
12 complete. time: 0.003796000000001243
13 complete. time: 0.005358999999998559
14 complete. time: 1.5906239999999983
15 complete. time: 0.02093099999999737
16 complete. time: 0.0058280000000010546
17 complete. time: 0.020697999999995886
18 complete. time: 0.006979999999998654
19 complete. time: 0.0037130000000047403
20 complete. time: 0.004607999999997503
21 complete. time: 0.006962999999998942
22 complete. time: 0.005015000000000214
23 complete. time: 0.003990000000001714
24 complete. time: 0.15149800000000369
25 complete. time: 